# API Exercises

Here are two exercises that involve working with APIs and dictionaries.

One is using the Open Brewery API found at https://www.openbrewerydb.org/, and the other is using the API for UK Police Data, found at https://data.police.uk/docs/.

You can complete them in either order!

Remember that you can create new cells with esc + a or b

## Breweries

### Q1: Load the first page of results with 50 results per page

In [5]:
import requests 
first_50 = requests.get('https://api.openbrewerydb.org/breweries?page=1&per_page=50')
first_50.json()
# url = 'https://api.openbrewerydb.org/breweries?page=1&per_page=50'

[{'id': 2,
  'name': 'Avondale Brewing Co',
  'brewery_type': 'micro',
  'street': '201 41st St S',
  'city': 'Birmingham',
  'state': 'Alabama',
  'postal_code': '35222-1932',
  'country': 'United States',
  'longitude': '-86.774322',
  'latitude': '33.524521',
  'phone': '2057775456',
  'website_url': 'http://www.avondalebrewing.com',
  'updated_at': '2018-08-23T23:19:57.825Z'},
 {'id': 4,
  'name': 'Band of Brothers Brewing Company',
  'brewery_type': 'micro',
  'street': '1605 23rd Ave',
  'city': 'Tuscaloosa',
  'state': 'Alabama',
  'postal_code': '35401-4653',
  'country': 'United States',
  'longitude': '-87.5621551272424',
  'latitude': '33.1984907123707',
  'phone': '2052665137',
  'website_url': 'http://www.bandofbrosbrewing.com',
  'updated_at': '2018-08-23T23:19:59.462Z'},
 {'id': 44,
  'name': 'Trim Tab Brewing',
  'brewery_type': 'micro',
  'street': '2721 5th Ave S',
  'city': 'Birmingham',
  'state': 'Alabama',
  'postal_code': '35233-3401',
  'country': 'United States

### Q2: This is only the first 50 results.  Get the next 50 and put them together.

In [83]:
second_50 = requests.get('https://api.openbrewerydb.org/breweries?page=2&per_page=50')
top_100 = first_50.json()+second_50.json()
top_100_test_lol = [first_50.json(),second_50.json()]
top_100_test_flat = [val for sublist in top_100_test_lol for val in sublist]
len(top_100_test_lol)
len(top_100_test_flat)


# is equivalent to:

# flattened = []
# for sublist in list_of_lists:
#     for val in sublist:
#         flattened.append(val)


100

### Q3: How many of these 100 breweries in are in Alaska?

In [38]:
full_list_of_states = list(map(lambda country: country['state'],top_100))
setlist_of_states_from_top_100 = list(set(full_list_of_states))
setlist_of_states_from_top_100
def breweries_in_state(unique_list_of_states, full_list):
    CF=[]
    for state in unique_list_of_states:
        CF_dict = {}
        CF_dict['state'] = state
        CF_dict['count'] = full_list.count(state)
        CF.append(CF_dict)
    #return CF_dict
    return sorted(CF, key=lambda k:k['count'],reverse=True)
breweries_in_state(setlist_of_states_from_top_100, full_list_of_states)

[{'state': 'California', 'count': 69},
 {'state': 'Arizona', 'count': 12},
 {'state': 'Arkansas', 'count': 5},
 {'state': 'Colorado', 'count': 5},
 {'state': 'Alabama', 'count': 4},
 {'state': 'Alaska', 'count': 3},
 {'state': None, 'count': 2}]

### Q4: Of these 100 breweries, what are the different unique brewery types?

In [27]:
full_list_of_brew_types = list(map(lambda typ: typ['brewery_type'],top_100))
setlist_of_brew_types_from_top_100 = list(set(full_list_of_brew_types))
setlist_of_brew_types_from_top_100

['brewpub', 'planning', 'proprietor', 'contract', 'regional', 'micro']

### Q5: What is the cloest brewery to "Devil's Potion Brewing Company LLC" ?
* Hint 1: Use Euclidian distance w/ longitude and latitude (assume longitude and latitude are a Carteisan coordinate system)
* Hint 2: You'll have to ignore the entries with "none" for latitude or longitude

In [59]:
test = list(map(lambda x: float(x['latitude']),top_100[0:2]))
test

[33.524521, 33.1984907123707]

In [61]:
def remove_invalid_distance(brewery_list):
    #remove long/lat values of "None"
    return list(filter(lambda x: x['latitude']!=None,brewery_list))

def convert_long_lat_to_float(brewery_list):
    #coerce latitude into float
    #coerce longitude into float
    new_brew_list = []
    for brewery in brewery_list:
        new_brew = brewery.copy()
        new_brew['longitude'] = float(brewery['longitude'])
        new_brew['latitude'] = float(brewery['latitude'])
        new_brew_list.append(new_brew)
    return new_brew_list

top_100_cleaned = convert_long_lat_to_float(remove_invalid_distance(top_100))
devil_potion = list(filter(lambda x: x['name']=="Devil's Potion Brewing Company LLC",top_100_cleaned))[0]
devil_potion

{'id': 525,
 'name': "Devil's Potion Brewing Company LLC",
 'brewery_type': 'planning',
 'street': '',
 'city': 'Escondido',
 'state': 'California',
 'postal_code': '92026-3187',
 'country': 'United States',
 'longitude': -117.0814849,
 'latitude': 33.1216751,
 'phone': '7605329091',
 'website_url': 'http://www.devilspotion.com',
 'updated_at': '2018-08-23T23:27:21.742Z'}

In [64]:
import math
def brewery_distance(first_brewery, second_brewery):
    a = first_brewery['longitude']-second_brewery['longitude']
    b = first_brewery['latitude']-second_brewery['latitude']
    c_squared = a**2+b**2
    return math.sqrt(c_squared)
brewery_distance(devil_potion,top_100_cleaned[0])

30.309840116145182

In [79]:
def closest_brew_buddy(first_brewery, brewery_list):
    remove_first_brewery = list(filter(lambda list_brewery: first_brewery!=list_brewery,brewery_list))
    all_brew_distances_sorted_close_far = sorted(list(map(lambda other_brewery: brewery_distance(first_brewery, other_brewery),remove_first_brewery)))
    closest_brewery = all_brew_distances_sorted_close_far[0]
    return list(filter(lambda other_brewery: closest_brewery==brewery_distance(first_brewery, other_brewery),brewery_list))[0]

In [80]:
closest_brew_buddy(devil_potion,top_100_cleaned)

{'id': 924,
 'name': 'Port Brewing Co / The Lost Abbey',
 'brewery_type': 'micro',
 'street': '155 Mata Way Ste 104',
 'city': 'San Marcos',
 'state': 'California',
 'postal_code': '92069-2983',
 'country': 'United States',
 'longitude': -117.149141,
 'latitude': 33.141537,
 'phone': '8009186816',
 'website_url': 'http://www.portbrewing.com',
 'updated_at': '2018-08-24T00:01:07.397Z'}

### Q6: Write a function to find the closest brewery to any other given brewery

In [ ]:
#see closest_brew_buddy()


### Q7: How would you get the first 10 pages from this API and put them all together using a for loop?

In [100]:
def pull_api_brewery_info(number_of_pages):
    brew_lol = []
    for page in list(range(1, number_of_pages+1)):
        brew_entry = requests.get('https://api.openbrewerydb.org/breweries?page='+str(page)+'&per_page=50')
        brew_lol.append(brew_entry.json())
    return [val for sublist in brew_lol for val in sublist]
len(pull_api_brewery_info(10))

500

# Crime in the UK

### We will be analyzing different crimes reported in the UK as provided by https://data.police.uk/docs/

# Exploratory analysis
##### 1. How many total crimes were there at latitude : 52.63902 and -1.131321 on November of 2017.
Use the street level crimes data, the documentation for the API can be found at https://data.police.uk/docs/method/crime-street/

##### 2. We've queried the API once, but it could get annoying to retype the url over and over again, create a function `make_api_request` that enables you to query the API.


 The parameters for the function should be:
* lat (float) : latitude
* lng (float) : longitude
* date (string): Date in the format YYYY-MM
    * default value = `None`
    
And it should return a json object of 

for more information on default values check out http://blog.thedigitalcatonline.com/blog/2015/02/11/default-arguments-in-python/

##### 3. Write a function `categories_of_crime` that will determine the count of each type of crime for a given latitude and longitude. This is labelled as 'category' in the records. Your function should call the `make_api_request` function you created.

The parameters for the function should be:

* lat (float) : latitude
* lng (float) : longitude
* date (str) default = None

The function should return:
* a dictionary with the count of each type of crime



Once you've created the function, try it with these locations
* lat, lng of 51.5017861,-0.1432319   (Buckingham Palace)
* lat, lng of 53.480161, -2.245163     (Manchester)

**Bonus**: 
* Write a function that determines the difference between Buckingham Palace and Manchester in terms of the number of crimes in each category.
    * In which category is there the largest absolute difference between the category of crime?
* Create a histogram depiction of the categories of crime

##### 4. Create a function `find_outcome_statuses` that will determine outcome statuses for a given latitude and longitude and date (optional)?
Investigate the data to determine where the outcome statuses are located.

**NOTE**: You'll notice that some of these crimes do not have crime outcomes. Make these into the category of "Not Resolved."

**NOTE 2**: These might take a long time to execute if you do not specify a month

**Bonus**: What is the ratio of crimes investigated to those not investigated? Is it higher near London or Manchester?

##### 5. Write a function `month_highest_crimes` that will return the month that had the highest number of crimes for a latitude, longitude and a year.

Inputs
* lat (float) : latitude
* lng (float) : longitude
* year (str) : in the format YYYY

Output
* month with highest crime (int)

**Bonus** Make a graph of how the number of crimes changed over time for a year. This will likely require a new function. Is seasonality a factor? Do the type of crimes change over time?

### Bonus Open Ended Questions

1. Take a look at the https://data.police.uk/docs/method/stops-street/ API. Is there a correlation between gender and being stopped and searched? How about race and being stopped and searched?